In [1]:
# import all libraries etc.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import warnings

In [31]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [45]:
data = pd.read_csv("data/Xtrain.csv")
y = pd.read_csv('data/ytrain.csv')

test = pd.read_csv('data/Xtest.csv')

In [3]:
data['price'] = y

In [4]:
data

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,3/4/2015,3,2.50,1880,4499,2.0,0.0,0.0,3,8,1880,0.0,1993,0.0,98029,47.5664,-121.999,2130,5114,529000.0
1,10/7/2014,3,2.50,2020,6564,1.0,0.0,0.0,3,7,1310,710.0,1994,0.0,98042,47.3545,-122.158,1710,5151,253000.0
2,1/16/2015,5,4.00,4720,493534,2.0,0.0,0.0,5,9,3960,760.0,1975,0.0,98027,47.4536,-122.009,2160,219542,745000.0
3,3/30/2015,2,2.00,1430,3880,1.0,0.0,0.0,4,7,1430,0.0,1949,0.0,98117,47.6844,-122.392,1430,3880,545000.0
4,10/14/2014,3,2.25,2270,32112,1.0,0.0,0.0,4,8,1740,530.0,1980,0.0,98042,47.3451,-122.094,2310,41606,390000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,9/15/2014,3,2.50,2230,5800,2.0,0.0,0.0,3,7,2230,0.0,2004,0.0,98065,47.5308,-121.847,2230,6088,440000.0
16193,10/2/2014,4,2.75,2770,3852,2.0,0.0,0.0,3,8,2770,0.0,2014,NaN,98178,47.5001,-122.232,1810,5641,572000.0
16194,7/21/2014,4,1.50,1530,9000,1.0,0.0,0.0,4,6,1530,0.0,1976,0.0,98014,47.6492,-121.908,1520,8500,299800.0
16195,6/20/2014,1,0.75,380,15000,1.0,0.0,0.0,3,5,380,0.0,1963,0.0,98168,47.4810,-122.323,1170,15000,245000.0


In [5]:
sqft_basement_list= []
for num in data['sqft_basement'].values:
    if num == '?':
        sqft_basement_list.append(np.nan)
    else:
        sqft_basement_list.append(float(num))
data['sqft_basement']= sqft_basement_list
#cleaning year renovated        
data['yr_renovated'] = data['yr_renovated'].fillna(0).astype(int)
data['yr_renovated'] = np.where(data['yr_renovated'] == 0, data['yr_built'], data['yr_renovated'])


#waterfron cleaning
data['waterfront'].replace(np.nan, 0, inplace =True)
#view col cleaning
data['view'].replace(np.nan, 0 , inplace =True)

#basement
data['sqft_basement'].fillna(0, inplace= True)

#drop not useful cols such as ID, Date
#data.drop(['id', 'date'],1, inplace= True)

#one house has a 33 bedroom house to 3:
data['bedrooms'].replace(33, 3, inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16197 entries, 0 to 16196
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           16197 non-null  object 
 1   bedrooms       16197 non-null  int64  
 2   bathrooms      16197 non-null  float64
 3   sqft_living    16197 non-null  int64  
 4   sqft_lot       16197 non-null  int64  
 5   floors         16197 non-null  float64
 6   waterfront     16197 non-null  float64
 7   view           16197 non-null  float64
 8   condition      16197 non-null  int64  
 9   grade          16197 non-null  int64  
 10  sqft_above     16197 non-null  int64  
 11  sqft_basement  16197 non-null  float64
 12  yr_built       16197 non-null  int64  
 13  yr_renovated   16197 non-null  int64  
 14  zipcode        16197 non-null  int64  
 15  lat            16197 non-null  float64
 16  long           16197 non-null  float64
 17  sqft_living15  16197 non-null  int64  
 18  sqft_l

In [21]:
def rmv_outlier (df, param):
    i = 0
    drop_index = []
    for x in df[param]:
        if x > df[param].mean() + 2*df[param].std():
            drop_index.append(i)
        if x < df[param].mean() - 2*df[param].std():
            #if x<=1:
            #    pass
            drop_index.append(i)
        i+=1
    df.drop(df.index[drop_index], axis =0, inplace = True)
    return df.reset_index()


In [ ]:
rmv_outlier(data,'price')

In [8]:
zip_code = data.groupby("zipcode")["price"].mean().sort_values(ascending = False)
zip_code.describe()

top_20 = data.groupby("zipcode")["price"].mean().sort_values(ascending = False)[:20]
top_20

zipcode
98039    2.192562e+06
98004    1.308105e+06
98040    1.225195e+06
98112    1.124661e+06
98102    9.022535e+05
98109    8.943315e+05
98105    8.878645e+05
98119    8.710023e+05
98006    8.499796e+05
98005    8.102966e+05
98033    7.997923e+05
98075    7.991353e+05
98199    7.793521e+05
98077    6.996543e+05
98074    6.817077e+05
98053    6.811292e+05
98177    6.633260e+05
98008    6.480832e+05
98122    6.427298e+05
98052    6.393276e+05
Name: price, dtype: float64

In [9]:
zipcode_index = top_20.index

In [10]:
data['top20_neighborhood'] = data['zipcode'].isin(zipcode_index).astype(int)

In [13]:
#Dummying grade
grade_origin= data['grade']
df_dummy_grade= pd.get_dummies(grade_origin, drop_first= True) #avoid dummy trap
data = pd.concat([df_dummy_grade, data], axis=1)

#renaming the new Columns
#grade
mapper = {4:'Grade_4', 5: 'Grade_5', 6:'Grade_6',7:'Grade_7',
          8:'Grade_8',9:'Grade_9',10:'Grade_10', 11:'Grade_11',12:'Grade_12',13:'Grade_13'} 
data.rename(mapper=mapper, axis=1, inplace= True)

In [14]:
#making dummy for condition
condition_origin= data['condition']
df_cond_dum=pd.get_dummies(condition_origin, drop_first=True)
data= pd.concat([df_cond_dum, data], axis=1)

mapper3 = {1:'Condition_1',2:'Condition_2',3:'Condition_3',4:'Condition_4',5:'Condition_5'}
data.rename(mapper=mapper3, axis=1, inplace=True)

In [15]:
#Dummying the view
view_origin= data['view']
df_dummy_view= pd.get_dummies(view_origin, drop_first= True)
data = pd.concat([df_dummy_view, data], axis=1)

#renaming the new Columns
#view

mapper_view = {4.0:'View 4', 2.0: 'View 2', 3.0:'View 3',1.0:'View 1'}

data.rename(mapper=mapper_view, axis=1, inplace= True)

In [18]:
# adding some interactions
data['age'] = 2021 - data['yr_built']

#decided to combine Bath and Bedroom into a ratio (Bed:Bath) since its a commonly used metric fpr real estate

data['bed:bath'] = data['bedrooms']/data['bathrooms']

data = data.loc[:,~data.columns.duplicated()]

In [19]:
# drop the original col that were used to dummy and those used for feature engineering

data.drop(['view', 'grade', 'bedrooms', 'bathrooms', 'condition', 'yr_renovated'], axis=1, inplace=True)

In [22]:
rmv_outlier(data,'price')

,index,View 1,View 2,View 3,View 4,Condition_2,Condition_3,Condition_4,Condition_5,Grade_4,...,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15,price,top20_neighborhood,age,bed:bath
0,0,0,0,0,0,0,1,0,0,0,...,1993,98029,47.5664,-121.999,2130,5114,529000.0,0,28,1.200000
1,1,0,0,0,0,0,1,0,0,0,...,1994,98042,47.3545,-122.158,1710,5151,253000.0,0,27,1.200000
2,2,0,0,0,0,0,0,0,1,0,...,1975,98027,47.4536,-122.009,2160,219542,745000.0,0,46,1.250000
3,3,0,0,0,0,0,0,1,0,0,...,1949,98117,47.6844,-122.392,1430,3880,545000.0,0,72,1.000000
4,4,0,0,0,0,0,0,1,0,0,...,1980,98042,47.3451,-122.094,2310,41606,390000.0,0,41,1.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14817,16192,0,0,0,0,0,1,0,0,0,...,2004,98065,47.5308,-121.847,2230,6088,440000.0,0,17,1.200000
14818,16193,0,0,0,0,0,1,0,0,0,...,2014,98178,47.5001,-122.232,1810,5641,572000.0,0,7,1.454545
14819,16194,0,0,0,0,0,0,1,0,0,...,1976,98014,47.6492,-121.908,1520,8500,299800.0,0,45,2.666667
14820,16195,0,0,0,0,0,1,0,0,0,...,1963,98168,47.4810,-122.323,1170,15000,245000.0,0,58,1.333333


In [24]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14822 entries, 0 to 16196
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   View 1              14822 non-null  uint8  
 1   View 2              14822 non-null  uint8  
 2   View 3              14822 non-null  uint8  
 3   View 4              14822 non-null  uint8  
 4   Condition_2         14822 non-null  uint8  
 5   Condition_3         14822 non-null  uint8  
 6   Condition_4         14822 non-null  uint8  
 7   Condition_5         14822 non-null  uint8  
 8   Grade_4             14822 non-null  uint8  
 9   Grade_5             14822 non-null  uint8  
 10  Grade_6             14822 non-null  uint8  
 11  Grade_7             14822 non-null  uint8  
 12  Grade_8             14822 non-null  uint8  
 13  Grade_9             14822 non-null  uint8  
 14  Grade_10            14822 non-null  uint8  
 15  Grade_11            14822 non-null  uint8  
 16  Grad

In [26]:
data = data.drop(['Grade_13', 'sqft_above', "yr_built"], axis=1)

In [27]:
data = data.drop('date', axis=1)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14822 entries, 0 to 16196
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   View 1              14822 non-null  uint8  
 1   View 2              14822 non-null  uint8  
 2   View 3              14822 non-null  uint8  
 3   View 4              14822 non-null  uint8  
 4   Condition_2         14822 non-null  uint8  
 5   Condition_3         14822 non-null  uint8  
 6   Condition_4         14822 non-null  uint8  
 7   Condition_5         14822 non-null  uint8  
 8   Grade_4             14822 non-null  uint8  
 9   Grade_5             14822 non-null  uint8  
 10  Grade_6             14822 non-null  uint8  
 11  Grade_7             14822 non-null  uint8  
 12  Grade_8             14822 non-null  uint8  
 13  Grade_9             14822 non-null  uint8  
 14  Grade_10            14822 non-null  uint8  
 15  Grade_11            14822 non-null  uint8  
 16  Grad

In [29]:
x_train = data.drop('price',axis = 1)
y_train = data['price']

In [32]:
x_train

,View 1,View 2,View 3,View 4,Condition_2,Condition_3,Condition_4,Condition_5,Grade_4,Grade_5,Grade_6,Grade_7,Grade_8,Grade_9,Grade_10,Grade_11,Grade_12,sqft_living,sqft_lot,floors,waterfront,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,top20_neighborhood,age,bed:bath
0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1880,4499,2.00,0.00,0.00,98029,47.57,-122.00,2130,5114,0,28,1.20
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2020,6564,1.00,0.00,710.00,98042,47.35,-122.16,1710,5151,0,27,1.20
2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,4720,493534,2.00,0.00,760.00,98027,47.45,-122.01,2160,219542,0,46,1.25
3,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1430,3880,1.00,0.00,0.00,98117,47.68,-122.39,1430,3880,0,72,1.00
4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,2270,32112,1.00,0.00,530.00,98042,47.35,-122.09,2310,41606,0,41,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2230,5800,2.00,0.00,0.00,98065,47.53,-121.85,2230,6088,0,17,1.20
16193,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,2770,3852,2.00,0.00,0.00,98178,47.50,-122.23,1810,5641,0,7,1.45
16194,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1530,9000,1.00,0.00,0.00,98014,47.65,-121.91,1520,8500,0,45,2.67
16195,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,380,15000,1.00,0.00,0.00,98168,47.48,-122.32,1170,15000,0,58,1.33


In [35]:
lr = LinearRegression()

In [37]:
scores_simple = cross_validate(
                    lr, x_train, y_train, cv=5, 
                    return_train_score=True)

In [41]:
lr.fit(x_train, y_train)
print(lr.score(x_train, y_train))

0.7242358097317736


In [42]:
y_hat_train = lr.predict(x_train)

In [43]:
train_mse = mean_squared_error(y_train, y_hat_train, squared =False)
print('Train Mean Error:', train_mse)

Train Mean Error: 97700.27350484233


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2/20/2015,3,0.75,850,8573,1.00,0.00,0.00,3,6,600,250.0,1945,0.00,98146,47.50,-122.36,850,8382
1,10/8/2014,3,1.00,1510,6083,1.00,0.00,0.00,4,6,860,650.0,1940,0.00,98115,47.70,-122.32,1510,5712
2,3/25/2015,4,2.25,1790,42000,1.00,0.00,0.00,3,7,1170,620.0,1983,0.00,98045,47.48,-121.74,2060,50094
3,2/17/2015,2,1.50,1140,2500,1.00,0.00,1.00,3,7,630,510.0,1988,nan,98106,47.57,-122.36,1500,5000
4,5/23/2014,3,1.00,1500,3920,1.00,0.00,0.00,3,7,1000,500.0,1947,0.00,98107,47.67,-122.36,1640,4017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,6/27/2014,5,1.00,1170,6757,1.00,0.00,0.00,4,6,800,370.0,1944,0.00,98125,47.73,-122.30,1590,6794
5396,1/20/2015,3,1.75,1670,5100,1.00,0.00,2.00,5,7,990,680.0,1954,0.00,98144,47.59,-122.29,2140,4452
5397,3/24/2015,4,2.25,3260,4640,2.00,0.00,0.00,5,9,2360,900.0,1907,0.00,98112,47.63,-122.31,3240,5800
5398,3/16/2015,4,5.00,5820,13906,2.00,0.00,0.00,3,11,3750,2070.0,1993,0.00,98042,47.38,-122.16,2980,13000
